# Similar Items
Add column to Items table to store similar items. We do this using KNN Algorithm and the store "embedding" vector for each item. We use the cosine similarity to find the similar items.

In [2]:
from supabase import create_client
from dotenv import load_dotenv
import os

load_dotenv("../.env.local")

client = create_client(
    os.getenv("SUPABASE_URL"),
    os.getenv("SUPABASE_KEY"),
)

print("Connected to Supabase")

Connected to Supabase


In [3]:
# Get items from database
result = client.table("items").select("*").execute()
print("Items from database:")
result

2024-01-13 18:49:10,067:INFO - HTTP Request: GET http://127.0.0.1:54321/rest/v1/items?select=%2A "HTTP/1.1 200 OK"


Items from database:


APIResponse[~_ReturnT](data=[{'id': 0, 'course_id': 1, 'author': 'Mary Hinsley', 'source_url': 'https://umich.instructure.com/courses/462614/files/21074771', 'file_name': 'BA 476 Handout Class 3 Reading.docx', 'number_pages': 19, 'content_name': ' "The Economic and Social Impact of Colonial Merchants, Slavery, and British Policies on American Business and the Revolutionary War"', 'content_summary': ' Colonial merchants played a crucial role in the economic, social, and political life of colonial America, despite facing challenges such as a lack of stable currency and banking systems. They engaged in various functions and had connections with influential figures in England. The need for labor was met through indentured servitude and slavery, with the slave trade becoming more profitable. The colonies also faced struggles with their currency and trade regulations imposed by Britain, leading to tensions and ultimately the Revolutionary War. The war brought about both challenges and opport

In [4]:
import ast
import numpy as np
from sklearn.neighbors import NearestNeighbors

dataset = result.data
embeddings = []
for row in dataset:
    embedding = row["embedding"]
    if isinstance(embedding, str):
        embedding = ast.literal_eval(embedding)
    embeddings.append(embedding)

neighbors = NearestNeighbors(n_neighbors=6).fit(embeddings)
nearest_neighbors = neighbors.kneighbors(embeddings, return_distance=False)
nearest_neighbors = nearest_neighbors[:, 1:]
nearest_neighbors

array([[12,  2, 13,  9, 11],
       [12,  2,  0,  5, 14],
       [12,  9,  0, 13, 14],
       [12, 13,  9,  2,  6],
       [ 7, 11, 17, 14, 13],
       [ 7,  2, 12,  4, 17],
       [13, 12,  3,  2,  0],
       [ 4, 17, 11, 13, 14],
       [16,  2,  9,  5, 12],
       [ 2, 12,  0, 11, 13],
       [14, 13, 17, 11, 12],
       [14, 13,  4, 17,  7],
       [ 2,  0, 13,  9, 14],
       [12, 14, 11,  0, 17],
       [13, 11, 10, 17,  4],
       [ 2, 10, 12, 14, 13],
       [ 8, 10,  2, 12,  9],
       [ 7,  4, 13, 14, 11]])

In [5]:
""" update rows with nearest neighbors """
nearest_neighbors = nearest_neighbors.tolist()
for i, row in enumerate(dataset):
    client.table("items").update({"nearest_neighbors": nearest_neighbors[i]}).eq("id", row["id"]).execute()

2024-01-13 18:49:17,717:INFO - HTTP Request: PATCH http://127.0.0.1:54321/rest/v1/items?id=eq.0 "HTTP/1.1 200 OK"
2024-01-13 18:49:17,734:INFO - HTTP Request: PATCH http://127.0.0.1:54321/rest/v1/items?id=eq.1 "HTTP/1.1 200 OK"
2024-01-13 18:49:17,747:INFO - HTTP Request: PATCH http://127.0.0.1:54321/rest/v1/items?id=eq.2 "HTTP/1.1 200 OK"
2024-01-13 18:49:17,758:INFO - HTTP Request: PATCH http://127.0.0.1:54321/rest/v1/items?id=eq.3 "HTTP/1.1 200 OK"


2024-01-13 18:49:17,770:INFO - HTTP Request: PATCH http://127.0.0.1:54321/rest/v1/items?id=eq.4 "HTTP/1.1 200 OK"
2024-01-13 18:49:17,782:INFO - HTTP Request: PATCH http://127.0.0.1:54321/rest/v1/items?id=eq.5 "HTTP/1.1 200 OK"
2024-01-13 18:49:17,795:INFO - HTTP Request: PATCH http://127.0.0.1:54321/rest/v1/items?id=eq.7 "HTTP/1.1 200 OK"
2024-01-13 18:49:17,808:INFO - HTTP Request: PATCH http://127.0.0.1:54321/rest/v1/items?id=eq.8 "HTTP/1.1 200 OK"
2024-01-13 18:49:17,818:INFO - HTTP Request: PATCH http://127.0.0.1:54321/rest/v1/items?id=eq.9 "HTTP/1.1 200 OK"
2024-01-13 18:49:17,826:INFO - HTTP Request: PATCH http://127.0.0.1:54321/rest/v1/items?id=eq.10 "HTTP/1.1 200 OK"
2024-01-13 18:49:17,835:INFO - HTTP Request: PATCH http://127.0.0.1:54321/rest/v1/items?id=eq.12 "HTTP/1.1 200 OK"
2024-01-13 18:49:17,843:INFO - HTTP Request: PATCH http://127.0.0.1:54321/rest/v1/items?id=eq.13 "HTTP/1.1 200 OK"
2024-01-13 18:49:17,853:INFO - HTTP Request: PATCH http://127.0.0.1:54321/rest/v1/ite